<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv4/blob/main/RBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/insertion_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/removal_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/adverserial_attacks_functions.py'
]

In [4]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 7.00MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 4.53MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 4.16MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 5.20MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 707kB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from adverserial_attacks_functions import *


torch.manual_seed(0)



cpu
cpu


In [6]:
# Open the .pkl file
with open('/content/insertion_array.pkl', 'rb') as f:
    # Load the object
    insertion_array = pickle.load(f)

# Close the file
f.close()

insertion_array = torch.tensor(insertion_array).to(device)
print(len(insertion_array))

# Open the .pkl file
with open('/content/removal_array.pkl', 'rb') as f:
    # Load the object
    removal_array = pickle.load(f)

# Close the file
f.close()

removal_array = torch.tensor(removal_array).to(device)
print(len(removal_array))


10000
10000


In [21]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [22]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 4096
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [9]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [10]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [11]:
# Create an instance of your model
model_AT_rFGSM_weightedLoss = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM_weightedLoss.load_state_dict(torch.load('model_AT_rFGSM_weightedLoss.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [12]:
import numpy as np
from sklearn.cluster import KMeans

# Function to initialize centers and sigma using K-Means clustering
def initialize_centers_sigma(data_loader, num_centers):
    # Collect data into a single tensor
    all_data = torch.cat([batch for batch, _ in data_loader], dim=0)
    all_data = all_data.numpy()

    #kmeans = KMeans(n_clusters=num_centers)
    kmeans = KMeans(n_clusters=num_centers, init='k-means++', n_init='auto')
    kmeans.fit(all_data)
    centers = kmeans.cluster_centers_

    # Calculate sigma based on the average distance between centers
    distances = np.sqrt(((centers[:, np.newaxis] - centers) ** 2).sum(axis=2))
    sigma = np.mean(distances)  # Using the mean distance as sigma

    return centers, sigma


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from sklearn.cluster import KMeans

# Define the RBF model
class RBFModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, init_centers, init_sigmas):
        super(RBFModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.centers = nn.Parameter(torch.Tensor(init_centers))
        self.sigmas = nn.Parameter(torch.Tensor(init_sigmas))

        # Linear layer for output
        self.linear = nn.Linear(hidden_dim, output_dim)

    def gaussian(self, x, c, sigma):
        return torch.exp(-torch.sum((x - c) ** 2 / (2 * sigma ** 2), dim=1))

    def forward(self, x):
        radial_out = torch.stack([self.gaussian(x, c, s) for c, s in zip(self.centers, self.sigmas)], dim=1)
        output = self.linear(radial_out)
        return output

# Function to initialize centers and sigmas using K-Means clustering
def initialize_centers_sigmas(data_loader, num_centers):
    # Collect data into a single tensor
    all_data = torch.cat([batch for batch, _ in data_loader], dim=0)
    print(all_data.shape)
    all_data = all_data.numpy()

    #kmeans = KMeans(n_clusters=num_centers)
    kmeans = KMeans(n_clusters=num_centers, init='k-means++', n_init='auto')
    kmeans.fit(all_data)
    centers = kmeans.cluster_centers_

    # Calculate sigma based on the average distance between centers
    distances = np.sqrt(((centers[:, np.newaxis] - centers) ** 2).sum(axis=2))
    sigma = np.mean(distances)  # Using the mean distance as sigma

    return centers, sigma

# Function to evaluate the model on a dataset
def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            output = model(batch_x)
            loss = criterion(output, batch_y)
            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            correct_predictions += (predicted == batch_y).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / len(data_loader.dataset)

    return avg_loss, accuracy

In [19]:
from sklearn.cluster import MiniBatchKMeans

# Function to initialize centers and sigmas using MiniBatchKMeans clustering with streaming approach
def initialize_centers_sigmas(data_loader, num_centers, batch_size=128):
    # Initialize MiniBatchKMeans with the specified number of clusters
    #kmeans = MiniBatchKMeans(n_clusters=num_centers, batch_size=batch_size)
    kmeans = MiniBatchKMeans(n_clusters=num_centers, batch_size=batch_size, init='k-means++', n_init='auto')

    # Accumulate data batch by batch
    for batch_x, _ in data_loader:
        batch_data = batch_x.numpy()
        kmeans.partial_fit(batch_data.reshape(len(batch_x), -1))  # Reshape batch data if needed

    # Get cluster centers
    centers = kmeans.cluster_centers_

    # Calculate sigma based on the average distance between centers
    distances = np.sqrt(((centers[:, np.newaxis] - centers) ** 2).sum(axis=2))
    sigma = np.mean(distances)  # Using the mean distance as sigma

    return centers, sigma


In [ ]:
# Assuming you have DataLoader named train_loader, validation_loader with batch size 128
input_dim = 10000  # Size of input binary vectors
hidden_dim = 1000   # Number of RBF units(centers)
output_dim = 2     # Binary classification


# Initialize centers and sigmas using K-Means clustering
init_centers, init_sigma = initialize_centers_sigmas(train_loader, hidden_dim)

In [ ]:
# Instantiate the RBF model with initialized centers and sigmas
model = RBFModel(input_dim, hidden_dim, output_dim, init_centers, init_sigma)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10


for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0

    for i, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct_predictions += (predicted == batch_y).sum().item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss}, Train Accuracy: {accuracy}')

    # Evaluate on validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
